In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import pm4py
import ocpa
import os
SCENARIOS = ['normal','insuff_approval','late_po_decision']
VARIANTSPERSCENARIO = ['with_connections','no_connections']
OUTPUT_FOLDER = 'generated'
LOG_SUFFIX = '.xmlocel'
TIME_ADJ_LOG_SUFFIX = '-timeadj_log.xmlocel'
DFG_PERF_SUFFIX = '-dfg_perf.svg'
DFG_FREQ_SUFFIX = '-dfg_freq.svg'
OCPN_SUFFIX = '-ocpn.svg'
PM4PY_OCPN_SUFFIX = '-pm4py_ocpn.svg'
FLAT_FOLDER = 'flattened'
EXPLODED_FOLDER = 'exploded'

In [ ]:
def adjust_timedeltas(ocel_filepath):
    ocel = pm4py.read_ocel(ocel_filepath)
    previous_timestamp = None
    previous_converted_timestamp = None;
    for idx, row in ocel.events.iterrows():
        current_timestamp : datetime = row['ocel:timestamp'].to_pydatetime()
        if previous_timestamp != None:
            time_difference = current_timestamp-previous_timestamp;
            # Calculation of converted difference
            if time_difference >= timedelta(seconds=2):
                converted_difference = (time_difference.total_seconds()-2)*5
            else:
                converted_difference = 0
                
            if converted_difference < 0:
                print(converted_difference)
            new_timestamp = pd.Timestamp(previous_converted_timestamp + timedelta(hours=converted_difference));
            ocel.events.at[idx,'ocel:timestamp'] = new_timestamp
            previous_converted_timestamp = new_timestamp.to_pydatetime()
        else:
            previous_converted_timestamp = current_timestamp
        previous_timestamp = current_timestamp
    return ocel;

In [ ]:
def minePNFromLog(log):
    net, initial_marking, final_marking = pm4py.algo.discovery.inductive.algorithm.apply(log)
    gviz = pm4py.visualization.petri_net.visualizer.apply(net,initial_marking,final_marking)
    return gviz

In [ ]:
ocels = dict()
time_adj_ocel = dict()
petri_net_perf = dict()
for scenario in SCENARIOS:
    for variant in VARIANTSPERSCENARIO:
        file_path = f"./{scenario}/{scenario}_{variant}{LOG_SUFFIX}";
        ocels[scenario] = pm4py.read_ocel(file_path)
        time_adj_ocel[scenario]  = adjust_timedeltas(file_path)

        # Create output folder
        output_path =  f"./{scenario}/{OUTPUT_FOLDER}_{variant}";
        if not os.path.exists(output_path):
            os.mkdir(output_path)
        # Save time adjusted OCEL
        adj_ocel_path  = f"./{output_path}/{scenario}{TIME_ADJ_LOG_SUFFIX}"
        pm4py.write_ocel( time_adj_ocel[scenario], adj_ocel_path)

        # Mine Petrinet with PM4PY
        # ocpn_pm4py = pm4py.discover_oc_petri_net(time_adj_ocel[scenario])
        # ocpn_pm4py_vis = pm4py.visualization.ocel.ocpn.visualizer.apply(ocpn_pm4py, parameters={"format": "svg"})
        # pm4py.visualization.ocel.ocpn.visualizer.save(ocpn_pm4py_vis,f"{output_path}/{scenario}{PM4PY_OCPN_SUFFIX}")

        # Mine Petri net with OCPA
        ocpa_df, ocpa_se  =  ocpa.objects.log.importer.ocel.factory.apply(adj_ocel_path)
        ocpn_ocpa = ocpa.algo.discovery.ocpn.algorithm.apply(ocpa_df)
        diag = ocpa.algo.conformance.token_based_replay.algorithm.apply(ocpn_ocpa, ocpa_df) # Replay on same event log for annotations
        ocpa_gviz = ocpa.visualization.oc_petri_net.factory.apply(ocpn_ocpa, diagnostics=diag, variant="annotated_with_diagnostics", parameters={"format": "svg", "act_count": True, "med_sojourn_time": True, "avg_sojourn_time": True, 'missing_token': True})
        ocpa.visualization.oc_petri_net.factory.save(ocpa_gviz,f"{output_path}/{scenario}{OCPN_SUFFIX}")

        # Mine DFGs
        ocdfg_freq = pm4py.discover_ocdfg(time_adj_ocel[scenario])
        ocdfg_freq_vis = pm4py.visualization.ocel.ocdfg.visualizer.apply(ocdfg_freq, parameters={"format": "svg",  "annotation": "frequency"})
        pm4py.visualization.ocel.ocdfg.visualizer.save(ocdfg_freq_vis,f"{output_path}/{scenario}{DFG_FREQ_SUFFIX}")
        
        ocdfg_perf = pm4py.discover_ocdfg(time_adj_ocel[scenario])
        ocdfg_perf_vis = pm4py.visualization.ocel.ocdfg.visualizer.apply(ocdfg_perf, parameters={"format": "svg",  "annotation": "performance"})
        pm4py.visualization.ocel.ocdfg.visualizer.save(ocdfg_perf_vis,f"{output_path}/{scenario}{DFG_PERF_SUFFIX}")
    
        # Prepare explosion of event log

        exploded_df, exploded_se  =  ocpa.objects.log.importer.ocel.factory.apply(adj_ocel_path)
        # Flatten event log on every attribute
        for object_type,item in time_adj_ocel[scenario].objects[time_adj_ocel[scenario].object_type_column].value_counts().iteritems():
            flattened = pm4py.ocel_flattening(time_adj_ocel[scenario],object_type)
            flattened_path = f"{output_path}/{FLAT_FOLDER}";
            if not os.path.exists(flattened_path):
                os.mkdir(flattened_path)
            # Save petri net from flat event log
            flat_gviz =  minePNFromLog(flattened)
            pm4py.visualization.petri_net.visualizer.save(flat_gviz,f"{flattened_path}/petri_net_{object_type}.png")

            # Also explode event log with ocpa
            # Remove events with no case id (i.e. events which are not associated with an element of the selected case object type)
            exploded_df_na_removed = exploded_df.dropna(subset=[object_type])
            exploded_xes = pm4py.format_dataframe(exploded_df_na_removed, case_id=object_type, activity_key='event_activity', timestamp_key='event_timestamp', timest_format=None)
            pm4py.write_xes(exploded_xes,f"{flattened_path}/exploded_log_{object_type}.xes")
            exploded_gviz =  minePNFromLog(exploded_xes)
            pm4py.visualization.petri_net.visualizer.save(exploded_gviz,f"{flattened_path}/exploded_petri_net_{object_type}.png")